In [ ]:
import gc
import pandas as pd
import numpy as np
import sklearn as skl
import seaborn as sns
import lightgbm as lgb
import torch
import scipy
import opendatasets as od
import math
import torchmetrics
import utils.lgbm as lgbm
import importlib
import joblib
import category_encoders
import utils.mlp as mlp
import utils.embedding_pipeline as embedding_pipeline
import utils.early_stopping as early_stopping
import utils.mlp_pipeline as mlp_pipeline
import pytorch_tabular

/usr/local/lib/python3.8/dist-packages/pytorch_tabular/models/mixture_density/mdn.py:25: UserWarning: Wandb not installed. WandbLogger will not work.
  warnings.warn("Wandb not installed. WandbLogger will not work.")


In [ ]:
import pickle
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [ ]:
#Kaggle api key: 9c2fc93aaaf8815d9fa8f2ceeeb57b6b
od.download("https://www.kaggle.com/c/avazu-ctr-prediction/")

ctr_data=pd.read_csv("./avazu-ctr-prediction/train.gz",nrows=1000000) #Using only the first 1million rows


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ritaleite
Your Kaggle Key: ··········


100%|██████████| 1.19G/1.19G [00:09<00:00, 138MB/s]



Extracting archive ./avazu-ctr-prediction/avazu-ctr-prediction.zip to ./avazu-ctr-prediction


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def find_optimal_recall(fpr,tpr,threshold,rate):
    optimal_index=np.argmin(abs(fpr-rate*np.ones(len(fpr))))
    print(threshold[optimal_index])
    return tpr[optimal_index]

In [ ]:
X=ctr_data.loc[:,'C1':'C21']
Y=ctr_data.click

del ctr_data

    
training_range=range(0,math.floor(len(Y)*0.6))
validation_range=range(math.floor(len(Y)*0.6),math.floor(len(Y)*0.8))
testing_range=range(math.floor(len(Y)*0.8),math.floor(len(Y)*1))

#Establish continuous and categorical features
cat_cols=X.columns
cont_cols=X.columns.difference(cat_cols)


xtrain=X.loc[training_range]
ytrain=Y.loc[training_range]
xtest=X.loc[testing_range]
ytest=Y.loc[testing_range]
xval=X.loc[validation_range]
yval=Y.loc[validation_range]

x_train,x_test,x_val=embedding_pipeline.aggregate_low_card_BAF(xtrain,xtest,xval,cat_cols)

del X
del Y
gc.collect()


26

# Parameter Tuning

In [ ]:
import lgbm
aux=importlib.reload(lgbm)
param_list=lgbm.lgbm_param_sampler(20,7,'cpu')
print(len(param_list))
print(param_list[0])
i=0
for params in param_list:
    model = lgbm.LGBMClassifier(n_jobs=10, **params)  # Instantiate LGBM Model.
        
    # Fit model to training data.
    model.fit(xtrain, ytrain, categorical_feature=categorical_features,eval_set=[(xval,yval)],early_stopping_rounds=5,eval_metric='loss')
    # Obtain predictions in test data.
    predictions = model.predict_proba(xtest)[:, 1]

    joblib.dump(model,'lgbm_3{}.pkl'.format(i))

20
{'boosting_type': 'goss', 'device': 'cpu', 'learning_rate': 0.07723198108458322, 'max_bin': 30, 'min_data_in_leaf': 72, 'num_iterations': 411, 'num_leaves': 412}
Fraud Rate for test set: 0.17409275761683593


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


iteration:  0 ------- Auc:  0.7283351698971195


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


iteration:  1 ------- Auc:  0.7038710257210177


...

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


iteration:  18 ------- Auc:  0.6956017059349454


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


iteration:  19 ------- Auc:  0.5183217716768651
Done!
Best param set: {'boosting_type': 'goss', 'device': 'cpu', 'learning_rate': 0.025874328322797666, 'max_bin': 256, 'min_data_in_leaf': 194, 'num_iterations': 1160, 'num_leaves': 295}
AUC score:  0.7443816861657959


(LGBMClassifier(boosting_type='goss', device='cpu',
                learning_rate=0.025874328322797666, max_bin=256,
                min_data_in_leaf=194, num_iterations=1160, num_leaves=295),
 {'boosting_type': 'goss',
  'device': 'cpu',
  'learning_rate': 0.025874328322797666,
  'max_bin': 256,
  'min_data_in_leaf': 194,
  'num_iterations': 1160,
  'num_leaves': 295},
 (array([0.        , 0.        , 0.        , ..., 0.99991928, 0.99997309,
         1.        ]),
  array([0.00000000e+00, 1.27648711e-04, 3.82946132e-04, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]),
  array([1.90312067, 0.90312067, 0.88772055, ..., 0.00211498, 0.00205294,
         0.00204345])))

In [ ]:
#Usar apenas para models mlp com label encoding
x1=xtrain
x2=xtest
for col in xtrain.select_dtypes(object).columns:
    
    le=skl.preprocessing.LabelEncoder()
    xtrain[col]=le.fit_transform(xtrain[col])
    xtest[col] = xtest[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
    le.classes_ = np.append(le.classes_, '<unknown>')
    xtest[col]=le.transform(xtest[col])
for col in xtrain.select_dtypes(float).columns:
    xtrain[col]=xtrain[col].fillna(np.average(xtrain[col]))
    xtest[col]=xtest[col].fillna(np.average(xtest[col]))

xtrain=xtrain.fillna(-1)
xtest=xtest.fillna(-1)

KeyboardInterrupt: ignored

# Categorical Encodings

In [ ]:
cat_cols=xtrain.columns
cont_cols=xtrain.columns.difference(cat_cols)

target_encoder=category_encoders.target_encoder.TargetEncoder(cols=cat_cols,handle_unknown='value',handle_missing='value')
xtrain_encoded=target_encoder.fit_transform(xtrain,ytrain)
xtest_encoded=target_encoder.transform(xtest)

C:\Users\ritas\anaconda3\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\ritas\anaconda3\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


# Categorical Embeddings

In [ ]:
from sklearn.model_selection import ParameterSampler
from scipy.stats import uniform

embedding_pipeline=importlib.reload(embedding_pipeline)
param_list=mlp.mlp_param_sampler(20, len(xtrain.columns),7,device)

#This dataset has no continuous features
normalization='None'
xtrain_aux_cont=torch.tensor([])
xval_aux_cont=torch.tensor([])

xtrain_aux_cat=mlp_pipeline.normalization_transform(x_train,'None',[])
xval_aux_cat=mlp_pipeline.normalization_transform(x_val,'None',[])
method='variational'


i=0
alphas=list(ParameterSampler({'alpha': uniform(loc=1.5, scale=3-1.5 )}, n_iter=20, random_state=np.random.RandomState(7)))

for params in param_list[i:]:
        dims=embedding_pipeline.get_emb_dim_var(x_train,cat_cols,alphas[i]['alpha'])
        dims=torch.tensor(dims,dtype=int).to(device)
        print(dims)
        # Fit pipeline
        print('iteration: ',i)
        
        model=embedding_pipeline.pipeline_no_cat(device,xtrain_aux_cat,xtrain_aux_cont,xval_aux_cat,xval_aux_cont,ytrain,yval,params,method,dims)

        #Save the model
        joblib.dump(model,'/content/gdrive/MyDrive/Colab Notebooks/data/mlp_3_emb/mlp_simple{}{}.pkl'.format(method,i))

        i=i+1

gc.collect()

tensor([[  10,    3],
        [   9,    3],
        [ 679,   40],
        [ 537,   35],
        [  21,    5],
        [ 630,   39],
        [  57,   10],
        [  23,    6],
        [ 825,   44],
        [8665,  148],
        [2053,   71],
        [   7,    2],
        [   7,    2],
        [ 405,   30],
        [   8,    3],
        [   9,    3],
        [ 145,   17],
        [   7,    2],
        [  43,    9],
        [ 146,   18],
        [  37,    8]])
iteration:  0
----------Method: Embedding for Categorical variational -------------
Train size:  600000 ; Number of 0:  501021 ; Number of 1: 98979
Val size:  200000 ; Number of 0:  168723 ; Number of 1: 31277
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping
tensor([[  10,    6],
        [   9,    6],
        [ 679,   68],
        [ 537,   60],
        [  21,   10],
        [ 630,   65],
        [ 

0

In [ ]:
import tabtransformer
tabtransformer=importlib.reload(tabtransformer)

cat_cols=['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id',
       'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
       'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21']

params=tabtransformer.tabtransformer_param_sampler(20,7,device)

for i in [5,12,17]:
  param=params[i]
  model=tabtransformer.transformer_pipeline(param,cat_cols,[],['click'],xtrain,xval,ytrain,yval,'gpu')
  joblib.dump(model,'/content/gdrive/MyDrive/aux/CTR_tabtransformer/tabtransformerCTR_{}'.format(i))
  predict_val=model.predict(xval)['1_probability']
  predict_test=model.predict(xtest)['1_probability']
  joblib.dump(predict_val,'/content/gdrive/MyDrive/aux/CTR_tabtransformer/yval_{}'.format(i))

  joblib.dump(predict_test,'/content/gdrive/MyDrive/aux/CTR_tabtransformer/yhat_{}'.format(i))



/usr/local/lib/python3.8/dist-packages/pytorch_tabular/models/tab_transformer/config.py:219: UserWarning: Ignoring the deprecated arguments, `out_ff_layers`, `out_ff_activation`, `out_ff_dropoout`, and `out_ff_initialization` as head_config is passed.
  warnings.warn(
2023-02-12 15:19:13,693 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_lite.utilities.seed:Global seed set to 42
2023-02-12 15:19:13,883 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-02-12 15:19:14,023 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
2023-02-12 15:19:17,054 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: TabTransformerModel
INFO:pytorch_t

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                   ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ TabTransformerBackbone │  541 K │
│ 1 │ _embedding_layer │ Embedding2dLayer       │ 16.7 M │
│ 2 │ _head            │ LinearHead             │ 18.1 M │
│ 3 │ loss             │ CrossEntropyLoss       │      0 │
└───┴──────────────────┴────────────────────────┴────────┘

Trainable params: 35.3 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 35.3 M                                                                                               
Total estimated model params size (MB): 141

Output()

2023-02-12 15:40:29,588 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
INFO:pytorch_tabular.tabular_model:Training the model completed
2023-02-12 15:40:29,592 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model
INFO:pytorch_tabular.tabular_model:Loading the best model


...

Output()

2023-02-12 16:22:16,443 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
INFO:pytorch_tabular.tabular_model:Training the model completed
2023-02-12 16:22:16,447 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model
INFO:pytorch_tabular.tabular_model:Loading the best model


5 epochs running time: 1576.9821319580078


Output()

Output()